In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Load data

Install `spacy` and download the raw data for the English and German Spacy tokenizers.  
NOTE: Administrator permission required. 
```bash
$ pip install spacy
$ python -m spacy download en
$ python -m spacy download de
```

In [14]:
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

SRC = Field(tokenize = "spacy",
            tokenizer_language="de",
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = True)

TRG = Field(tokenize = "spacy",
            tokenizer_language="en",
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = True)

train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), 
                                                    fields = (SRC, TRG), 
                                                    root='data/')
print(train_data[0].src)
print(train_data[0].trg)

['zwei', 'junge', 'weiße', 'männer', 'sind', 'im', 'freien', 'in', 'der', 'nähe', 'vieler', 'büsche', '.']
['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.']


In [40]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

# A dict mapping word to index
print(list(SRC.vocab.stoi.keys())[:5])
print(list(SRC.vocab.stoi.values())[:5])
# A list mapping index to word
print(SRC.vocab.itos[:5])

['<unk>', '<pad>', '<sos>', '<eos>', '.']
[0, 1, 2, 3, 4]
['<unk>', '<pad>', '<sos>', '<eos>', '.']


In [31]:
{1:2}

{1: 2}